# Using OTIs within OTIs A.K.A. 'OTI-Inception'
# Application: Total derivatives with OTI numbers.
```
Author: Mauricio Aristizabal, PhD
Date: 06/20/2023
```

The goal of this document is to show different approaches to obtain total derivatives with OTI numbers, in different applications. One of the approaches being evaluating an OTI with another OTI number.

For this, first import the library:

In [1]:
import pyoti.sparse as oti
import numpy as np
oti.set_printoptions(terms_print=-1)

# 1. Example 1

Consider the function 

$$
f(x,y) = x \sin(y)
$$

## 1.1 Partial derivatives
The partial derivatives of this function with respect to x and y are as follows:


$$
\begin{array}{rcl}
\partial f/\partial x &= &\sin(y) \\
\partial f/\partial y &= &x\cos(y) \\
\end{array}
$$
and its second order partial derivatives are as follows:

$$
\begin{array}{rcl}
\partial^2 f/\partial x^2  &= &0 \\
\partial^2 f/\partial x\partial y &= &\cos(y) \\
\partial^2 f/\partial y^2 &= &-x\sin(y) \\
\end{array}
$$

In order to compute these with OTIs, we simply define apply independent imaginary perturbations to each variable. Lets consider $x=1.5$ and $y=2.25$

In [2]:
def func(x,y):
    return x * oti.sin(y)


In [3]:
x = 1.5+oti.e(1,order=2)
y = 2.25+oti.e(2,order=2)

In [4]:
f1 = func(x,y)
f1

1.16711 + 0.778073 * e([1]) - 0.94226 * e([2]) - 0.628174 * e([1,2]) - 0.583555 * e([[2,2]])

Lets compare the results with the analytical solution values

In [5]:
f1.get_deriv(1), np.sin(y.real)

(0.7780731968879212, 0.7780731968879213)

In [6]:
f1.get_deriv(2), x.real * np.cos(y.real)

(-0.9422604340841088, -0.9422604340841085)

In [7]:
f1.get_deriv([1,1]), 0

(0.0, 0)

In [8]:
f1.get_deriv([1,2]), np.cos(y.real)

(-0.6281736227227391, -0.628173622722739)

In [9]:
f1.get_deriv([2,2]), -x.real*np.sin(y.real)

(-1.1671097953318819, -1.167109795331882)

## 1.2 Total derivatives
Now consider the case $y$ is a function of x, say $y(x)=x^2$. The total derivative of $f$ with respect to $x$ is defined as:

$$
\frac{Df}{Dx} = \frac{\partial f}{\partial x} +  \frac{\partial f}{\partial y}\frac{\partial y}{\partial x}
$$
for the particular ase of $f(x,y)=x\sin(y)$ 
$$
\frac{Df}{Dx}=\sin(y)+2x^2\cos(y) 
$$



The second order partial derivative is as follows
$$
\frac{D^2f}{Dx^2} = \frac{\partial^2 f}{\partial x^2} +
2\frac{\partial^2 f}{\partial x\partial y}\frac{\partial y}{\partial x}+
\frac{\partial^2 f}{\partial y^2}\left(\frac{\partial y}{\partial x}\right)^2+
\frac{\partial f}{\partial y}\frac{\partial^2 y}{\partial x^2}
$$
and for the function
$$
\frac{D^2f}{Dx^2} = 6x\cos(y)-4x^3\sin(y)
$$

There are multiple ways to find this total derivatives with OTIs. 

**The most straight forward way** is if the function $y(x)$ is explicitly known prior to the computation of $f(x,y)$, then we can evaluate y at the OTI-perturbed value of x, i.e. $y(x+\epsilon_1)$.

In [10]:
y2 = x**2
y2

2.25 + 3 * e([1]) + 1 * e([[1,2]])

After this, we can use the new perturbed value of $y$ to compute the total derivative of $f$ with respect to $x$.

In [11]:
f2 = func(x,y2)
f2

1.16711 - 2.04871 * e([1]) - 8.07878 * e([[1,2]])

Comparing this with the analytical values, we obtain:

In [12]:
f2.get_deriv(1),np.sin(y.real)+2*x.real**2*np.cos(y.real)

(-2.048708105364405, -2.0487081053644047)

In [13]:
f2.get_deriv([1,1]),6*x.real*np.cos(y.real)-4*x.real**3*np.sin(y.real)

(-16.15755076249159, -16.15755076249159)

**An alternative way** to compute the total derivative is to reuse the partial derivatives computed in the first run in a Taylor series, and evaluate such taylor series in the now known function $y(x)$. 

That is, evaluate the surrogate of the function at the values of the now known derivatives of $y$ with respect to x. For this we need:
1. The new OTI-form of y with respect to x (this is in ```y2```).
$$
y(x+\epsilon_1) = y+\frac{dy}{dx}\epsilon_1 + \frac{1}{2}\frac{dy}{dx}\epsilon_1^2
$$
2. The expansion of partial derivatives of f (see section 1.1)

We use the ```.rom_eval_object(imDirList, varDeltaList)``` in order to evaluate the Taylor series along the correspondign values. From the first OTI result in ```f1```, we know that $\epsilon_1$ is associated to $x$ and $\epsilon_2$ associated to $y$. 

We dont want to change anything with respect to $x$ at this point, so we neet to "replace" $\epsilon_1$ by $\epsilon_1$ with the same truncation conditions. What we want is to transform the terms of $y$ by the new known relation, thus we want to transform the $\epsilon_2$ (derivatives with respect to $y$) by the new derivatives known from $y(x+\epsilon_1)$, that is 

$$\Delta y = y(x+\epsilon_1) - y = \frac{dy}{dx}\epsilon_1 + \frac{1}{2}\frac{dy}{dx}\epsilon_1^2 $$

So now we evaluate the first function result using the following command

```f1.rom_eval_object([ 1, 2],[oti.e(1,order=2),y2-y.real])```

In [14]:
f3 = f1.rom_eval_object([ 1, 2],[oti.e(1,order=2),y2-y.real])
f3

1.16711 - 2.04871 * e([1]) - 8.07878 * e([[1,2]])

In [15]:
f3.get_deriv(1),np.sin(y.real)+2*x.real**2*np.cos(y.real)

(-2.048708105364405, -2.0487081053644047)

In [16]:
f3.get_deriv([1,1]),6*x.real*np.cos(y.real)-4*x.real**3*np.sin(y.real)

(-16.15755076249159, -16.15755076249159)